In [ ]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(
        p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
pprint([str(p) for p in file_abs_paths])

資料夾絕對路徑:
/workspaces/robert/lesson19/每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
['/workspaces/robert/lesson19/每日各站進出站人數/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/robert/lesson19/每日各站進出站人數/每日各站進出站人數2020.csv',
 '/workspaces/robert/lesson19/每日各站進出站人數/每日各站進出站人數2021.csv',
 '/workspaces/robert/lesson19/每日各站進出站人數/每日各站進出站人數2022.csv',
 '/workspaces/robert/lesson19/每日各站進出站人數/每日各站進出站人數2023.csv']


下面以步驟化方式說明這個儲存格的程式碼，並給出注意事項與可選的改善版本。

高階概覽
- 這個小函式會在工作目錄下尋找名為「每日各站進出站人數」的資料夾，列出其中所有 CSV 檔案的絕對路徑，並排除 manifest.csv 與 schema.csv，最後回傳該資料夾的絕對路徑與 CSV 檔案清單。程式最後示範呼叫並把結果印出來。

逐行說明
- from pathlib import Path
  - 使用 pathlib 提供跨平台的路徑物件 API（Path）。
- from pprint import pprint
  - 引入 pprint 以漂亮方式輸出列表（更好閱讀）。
- def list_daily_flow_files(base_dir: Path | None = None):
  - 定義函式；參數 base_dir 預設為 None，並使用 Python 3.10+ 的 union 型別註記 (Path | None)。
- docstring
  - 三行描述要做的事（中文）。
- if base_dir is None:
    base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
  - 若未提供 base_dir，則以目前工作目錄 Path.cwd() 為基底。
- target_dir = base_dir / "每日各站進出站人數"
  - 使用 Path 的 / 運算子拼接子目錄路徑（等同於 Path(base_dir, "...")）。
  - 假設 base_dir 已是 Path（呼叫端若傳 str 會出錯，見改進建議）。
- if not target_dir.is_dir():
    raise FileNotFoundError(f"找不到資料夾: {target_dir}")
  - 若該子資料夾不存在或不是目錄，丟出例外提醒。
- excluded = {"manifest.csv", "schema.csv"}
  - 要排除的檔名集合（以檔名字串比較）。
- csv_paths = sorted(
      p.resolve()
      for p in target_dir.glob("*.csv")
      if p.name not in excluded
  )
  - target_dir.glob("*.csv")：列出該目錄底下與模式匹配的檔案（只搜尋該層，不遞迴）。
  - 以 p.name 和 excluded 做比對，若名字不在排除集合才保留。
  - p.resolve()：取得絕對（解析過的）路徑。
  - sorted(...)：把結果按字典序排序並回傳為 list。
- return target_dir.resolve(), csv_paths
  - 回傳資料夾的絕對路徑（Path）與 CSV 路徑列表（List[Path]）。
- folder_abs_path, file_abs_paths = list_daily_flow_files()
  - 實際呼叫函式，使用預設工作目錄。
- print(...) 與 pprint([...])
  - 印出資料夾絕對路徑與檔案清單（pprint 以字串形式列印路徑）。

可能的 gotchas（容易被忽略的行為）
- base_dir 若被呼叫者傳入為 str，直接用 base_dir / "..." 會失敗；建議在函式內統一轉成 Path。
- excluded 比對是大小寫敏感的（以 p.name 作精確比對）。在 Linux 上檔名大小寫敏感，在 Windows/NTFS 可能是大小寫不敏感，行為會隨平台不同。
- glob("*.csv") 為非遞迴（不會搜尋子目錄）。若需要包含子目錄，應改用 rglob("*.csv")。
- Path.resolve() 會解析 symlink；在某些情況或舊 Python 版本中若檔案不存在可能會拋例外（可用 strict=False 來避免）。
- 程式使用了 Python 3.10 的型別語法（Path | None），在較舊 Python 版本會語法錯誤。

簡短改進建議與範例實作
- 讓 base_dir 接受 str 或 Path，並在函式內轉成 Path。
- 將 excluded 的比對改為不區分大小寫（視需求）。
- 支援遞迴選項、或讓 excluded 可由外部傳入以提高靈活性。

改進後的範例（可直接替換或加入為新函式）：



In [ ]:
# ...existing code...
def list_daily_flow_files(
    base_dir: Path | str | None = None,
    excluded_names: set[str] | None = None,
    recursive: bool = False,
):
    """
    取得「每日各站進出站人數」資料夾下的 CSV 檔案清單。
    - base_dir: Path 或 str；若為 None 則用 Path.cwd()
    - excluded_names: 要排除的檔名集合（比對時不區分大小寫）
    - recursive: 是否遞迴搜尋子目錄（True 使用 rglob）
    回傳: (target_dir: Path (resolved), csv_paths: list[Path])
    """
    if base_dir is None:
        base_dir = Path.cwd()
    base_dir = Path(base_dir)  # 接受 str 或 Path

    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    if excluded_names is None:
        excluded_names = {"manifest.csv", "schema.csv"}
    excluded_lower = {n.lower() for n in excluded_names}

    pattern = "**/*.csv" if recursive else "*.csv"
    globber = target_dir.rglob(pattern) if recursive else target_dir.glob(pattern)

    csv_paths = sorted(
        (p.resolve() for p in globber if p.name.lower() not in excluded_lower),
        key=lambda p: p.name.lower()
    )
    return target_dir.resolve(), csv_paths
# ...existing code...



使用範例與說明
- 若只要當前目錄下非遞迴行為（與原本相同）：
  - folder_abs_path, file_abs_paths = list_daily_flow_files()
- 若希望遞迴且不區分大小寫排除：
  - folder_abs_path, file_abs_paths = list_daily_flow_files(recursive=True)

結論（重點摘要）
- 原程式簡潔明確地達成「列出指定資料夾下 CSV，排除兩個檔案」的目的。
- 可改進處主要在於：接受 str、大小寫敏感問題、是否遞迴、以及讓排除名單可配置。

In [5]:
import pandas as pd
current_dir = Path.cwd()
# 若 CSV 與此 notebook 同資料夾
csv_path = current_dir / "台鐵車站資訊.csv"
stations_df = pd.read_csv(csv_path)
#display(stations_df.head())
stations_df = stations_df.reindex(columns=["stationCode", "stationName"])

#欄位名稱更改為[車站代碼, 車站名稱]
stations_df.columns = ["車站代碼", "車站名稱"]
stations_df

,車站代碼,車站名稱
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [6]:
#建立一個function
#要concate下面迴圈的所有merged_df
def process_yearly_data(file_abs_paths, stations_df):
    merged_dfs = []
    for csv_path in file_abs_paths:
        year_df = pd.read_csv(csv_path)
        year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
        #display(year_df.head())
        #日期欄位目前是int64, 需要轉換為datetime格式
        year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
        merged_df = pd.merge(year_df, stations_df, on="車站代碼")
        merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
        merged_df.head()
        #將欄位:日期,變為index
        merged_df.set_index("日期", inplace=True)
        merged_dfs.append(merged_df)
    return pd.concat(merged_dfs)
result_df = process_yearly_data(file_abs_paths, stations_df)
result_df

,車站名稱,進站人數,出站人數
日期,,,
2019-04-23,基隆,8442,7743
2019-04-23,三坑,1394,1348
2019-04-23,八堵,2770,2423
2019-04-23,七堵,6113,6335
2019-04-23,百福,2680,2726
...,...,...,...
2023-12-31,瑞芳,7916,8252
2023-12-31,海科館,164,195
2023-12-31,八斗子,652,720


以下以繁體中文、Markdown 格式對該 code cell（process_yearly_data 與後續呼叫）的程式碼做詳細說明、逐行解析與注意事項。

高階概述
- 功能：讀取多個每日進出站 CSV（file_abs_paths），把每個檔案的日期轉成 datetime、與 stations_df（車站對照表）合併，整理欄位並把日期設為 index，最後把所有年度的 DataFrame 串接成一個結果 result_df。
- 前提變數：
  - file_abs_paths：由前面 list_daily_flow_files 回傳的 CSV 絕對路徑清單（list[Path]）。
  - stations_df：先前由台鐵車站資訊 CSV 讀入並將欄位改為 ["車站代碼","車站名稱"] 的 DataFrame。

逐行說明
- def process_yearly_data(file_abs_paths, stations_df):
  - 定義函式，接受檔案路徑清單與車站對照表。
- merged_dfs = []
  - 初始化 list，用來累積每個檔案處理後的 DataFrame。
- for csv_path in file_abs_paths:
  - 迭代每一個 CSV 檔路徑（csv_path 可能是 pathlib.Path 或字串，pd.read_csv 可接受）。
- year_df = pd.read_csv(csv_path)
  - 讀取該 CSV 成為 DataFrame。預設不解析日期、不指定欄位名稱，讀入時欄位順序必須與下方重命名一致。
- year_df.columns = ["日期", "車站代碼", "進站人數", "出站人數"]
  - 直接覆寫欄位名稱（假設 CSV 有剛好 4 欄且順序固定）。若欄位數或順序不同，會丟錯誤或造成錯誤欄位對應。
- #display(year_df.head()) / #日期欄位目前是int64, 需要轉換為datetime格式
  - 註解說明：原始日期是數字（例如 20200101），需要轉 datetime。
- year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d")
  - 把「日期」欄轉為 datetime dtype，使用指定格式加速與避免猜測錯誤。若有非符合格式的值會丟例外（可加 errors="coerce"）。
- merged_df = pd.merge(year_df, stations_df, on="車站代碼")
  - 以車站代碼為鍵，將 year_df 與 stations_df 做內部合併（inner join）。合併結果包含年表所有欄位與車站名稱。注意兩邊車站代碼型別需一致（整數 vs 字串）否則可能找不到比對到的列。
- merged_df = merged_df.reindex(columns=["日期","車站名稱","進站人數","出站人數"])
  - 重新選取並按新順序排列欄位，去掉合併後不需要的欄（例如原始 stationCode 如果仍存在會被移除）。
- merged_df.head()
  - 呼叫 head() 但沒有賦值或顯示；在迴圈中這行沒效果（非 cell 最末行時不會顯示）。可移除或改為 display(merged_df.head())。
- #將欄位:日期,變為index
- merged_df.set_index("日期", inplace=True)
  - 把「日期」欄設為 index（就地修改）。結果 index 為 datetime index，方便時序操作。
- merged_dfs.append(merged_df)
  - 把每個年度處理好的 DataFrame 加入列表。
- return pd.concat(merged_dfs)
  - 把所有年度的 DataFrame 串接（沿著 index 與欄位合併，預設 axis=0）。若 merged_dfs 為空會引發 ValueError，且未指定 sort 或 ignore_index，回傳的 index 可能不是連續。若不同年度有重複 index（同一天不同檔案或不同車站），會保留多列。

呼叫與輸出
- result_df = process_yearly_data(file_abs_paths, stations_df)
  - 執行函式並把合併後結果指定給 result_df。
- result_df
  - notebook 中此行會顯示 result_df（若是 cell 的最後一行）。

可能的問題 (gotchas)
- pd.read_csv 讀入欄位數或順序與預期不符會導致 columns 指定失敗。
- 日期轉換：若有缺失或不合格式值，pd.to_datetime(..., format=...) 會丟錯（可用 errors="coerce" 轉成 NaT）。
- 合併鍵類型不一致（例如 stations_df 的車站代碼是字串但 year_df 是 int）會造成合併結果缺失。
- pd.concat 在 merged_dfs 為空時會丟 ValueError。若資料集可能為空，應處理例外或回傳 empty DataFrame。
- 記憶體：若 CSV 很大，將所有年度一次讀入並 concat 可能耗盡記憶體。可用 chunked 處理或逐步寫入 parquet/csv。
- merged_df.head() 在迴圈中沒有實際顯示效果，應移除或改用 display()。

建議的改進（短列舉）
- 在讀檔時指定 dtype 與 usecols，提高穩定性與效能：
  - pd.read_csv(csv_path, usecols=[0,1,2,3], dtype={"車站代碼": str})
- 處理空列表：
  - if not merged_dfs: return pd.DataFrame(columns=["車站名稱","進站人數","出站人數"])
- 處理日期轉換錯誤：
  - year_df["日期"] = pd.to_datetime(year_df["日期"], format="%Y%m%d", errors="coerce")
- 確保合併鍵同型別：
  - year_df["車站代碼"] = year_df["車站代碼"].astype(stations_df["車站代碼"].dtype)
- 若資料量大，改成逐檔處理並 append 到磁碟或使用 chunksize，再用分塊合併以節省記憶體。
- 若需要去重或排序 index：在最後加上 result = pd.concat(...).sort_index().drop_duplicates() 或 pd.concat(..., ignore_index=True)（視需求）。

總結（重點）
- 此 cell 主要流程：讀 CSV → 改欄名 → 轉 datetime → 以車站代碼合併車站名稱 → 重排欄位 → 設日期為 index → 把多個年度結果 concat。
- 注意欄位對應、日期轉換錯誤、合併鍵類型、空列表與記憶體使用，可依需求加入型別檢查、錯誤處理與記憶體優化。